In [1]:
import numpy as np
import openrouteservice
from openrouteservice.geocode import pelias_search
import xarray as xr
from datetime import datetime
from datetime import time as dtime
from datetime import date as ddate

ORS_API_KEY = "5b3ce3597851110001cf62485c01b629a61e4f57b5449ab3a0000ec5"

def get_temp_series(location = "Bergheimer Straße 116, 69115 Heidelberg, Germany"):
    """
    Given a location returns an xarray dataset of hourly temperatures in Kelvin
    INPUTS:
        something
    OUTPUTS:
        ds (xarray dataset): has coordinates [longitude (float32), latitude (float32), time (datetime64)]
        
    TODO: specify time bounds, properly comment
    """
    clnt = openrouteservice.Client(key=ORS_API_KEY)
    call = pelias_search(clnt, location)
    for feature in call["features"]:
        geom = feature["geometry"]["coordinates"]

    fn = "temp_data.nc"
    ds = xr.open_dataset(fn)
    #yearmonth = "2021-10-31"
    #yearmonth = datetime.strptime(yearmonth, '%Y-%m-%d')
    ds = ds.sel(latitude=round(geom[1])).sel(longitude=round(geom[0]))

    # interpolate missing hours of the day (sometimes data only from 06h to 22h)
    ds = ds.resample(time="1h").interpolate("linear")
    return ds


def mask_temp(data, Tref = 288.65):
    """
    INPUTS:
        data (xarray dataset): Must have datetimes in ["time"].values and temperatures in ["t2m"].values
        Tref (float): Reference temperature, given in Kelvin. Default is 291.15 (18 Celcius).
    OUTPUTS:
        something
    Note that for now I choose not to use heating start and stop times (ex. setting temp = Tref outside this range)
    becuase it adds complications, ex. the building will maintain some of the heating even if it is off... I think
    that turning the heating off will result in lower gas usage anyway... idk should talk about it
    """
    pass

    # hourly time series
    # compute degree days

def get_month_endpoints(date_str):
    """
    Given a year-month string, return the endpoints required for the timeslice to select temperatures of a given month
    INPUTS:
        date_str (str): should be 'year-month', ex. '2020-02'
    OUTPUTS:
        start_str (str): string for first day of month, ex. '2020-02-01'
        end_str (str): string for first day of next month at midnight, so as to include 11pm to midnight of 
                        last day, ex. '2020-03-01-00'
    """
    start_year , start_month = date_str.split('-')
    if int(start_month) < 12:
        end_month = '{:02d}'.format(int(start_month)+1)
        end_year = start_year
    elif int(start_month) == 12:
        end_month = '01'
        end_year = str(int(start_year)+1)
    else:
        raise Exception('Error in get_month_endpoints: month can not be >12!')

    start_str = start_year + '-' + start_month + '-01'
    end_str = end_year + '-' + end_month + '-01-00'
    return start_str, end_str

def calc_degreedays(type, date_str, Tref=288.65):
    """ Docstring TODO """
    if type not in ('heating','cooling'):
        print("WARNING: degree days type not understood. Defaulting to type='heating'.")
        type = 'heating'
    
    ds = get_temp_series() 
    
    month_ds = ds.sel(time=slice(*get_month_endpoints(date_str))) # this takes from midnight of first day to midnight of first day of following month
    diff_time = month_ds['time'][-1] - month_ds['time'][0] # get time difference, saved as xarray with np.timedelta64 array in values

    if type == 'heating':
        month_ds['t2m'] = Tref - month_ds['t2m']
    else: # type == cooling
        month_ds['t2m'] = month_ds['t2m'] - Tref
    month_ds = np.maximum(month_ds, 0)

    # integrate
    # note: integrate sets each hour as unit 1, must divide by total number of hours to obtain cumulative degree days
    dd = month_ds.integrate('time',datetime_unit='h') /24 #/ np.timedelta64(diff_time.values,'h').astype(float)
    #TODO: I believe the correct rescaling is 24, not total number of hours???
    
    return dd['t2m'].values

def heating_func(Tmax, Tmin, Tavg, Tref=None):
    ''' Doctring: TODO. Function from paper (reference)'''
    if Tmax != Tmin: #TODO: make sure this assumption ok
        assert ((Tmax > Tavg) and (Tavg > Tmin)), 'heating_func: Should have Tmax > Tavg > Tmin'
    if Tref >= Tmax:
        val = Tref - Tavg
    elif (Tavg <= Tref): # implicitly includes (Tref < Tmax)
            val = (Tref - Tmin)/2 - (Tmax - Tref)/4
    elif (Tmin <= Tref): # implicitly includes (Tref < Tavg < Tmax)
        val = (Tref - Tmin)/4
    elif (Tref <= Tmin):
        val = 0
    else:
        raise Exception('heating_func: something went wrong, should not get here')
    return val

def cooling_func(Tmax, Tmin, Tavg, Tref=295.15):
    ''' Doctring: TODO. Function from paper (reference)'''
    if Tmax != Tmin: #TODO: make sure this assumption ok
        assert ((Tmax > Tavg) and (Tavg > Tmin)), 'cooling_func: Should have Tmax > Tavg > Tmin'
    if Tref >= Tmax:
        val = 0
    elif (Tavg <= Tref): # implicitly includes (Tref < Tmax)
        val = (Tmax - Tref)/4
    elif (Tmin <= Tref): # implicitly includes (Tref < Tavg < Tmax)
        val = (Tmax - Tref)/2 - (Tref - Tmin)/4
    elif (Tref <= Tmin):
        val = Tavg - Tref
    else:
        raise Exception('cooling_func: something went wrong, should not get here')
    return val

heating_func_vec = np.vectorize(heating_func)
cooling_func_vec = np.vectorize(cooling_func)

def calc_degreedays2(type, date_str, Tref=288.65):
    # Tref = 288.65 for 15.5 C
    # Tref = 291.15 for 18 C
    # Tref = 295.15 for 22 C
    # Uses the methodology from the paper 'European degree-day climatologies and trends for the period 1951 – 2011', 
    # J. Spinoni, J. Vogt, and P. Barbosa. INTERNATIONAL JOURNAL OF CLIMATOLOGY, 2014.

    if type not in ('heating','cooling'):
        print("WARNING: degree days type not understood. Defaulting to type='heating'.")
        type = 'heating'

    ds = get_temp_series() 
    month_ds = ds.sel(time=slice(*get_month_endpoints(date_str))) # this takes from midnight of first day to midnight of first day of following month
    max_temps = month_ds.resample(time='1D').max()
    min_temps = month_ds.resample(time='1D').min()
    mean_temps = (max_temps + min_temps) / 2    

    if type == 'heating':
        if Tref == None: Tref = 288.65 # (15.5 C)
        dd = np.sum (xr.apply_ufunc( heating_func_vec, max_temps['t2m'], min_temps['t2m'], mean_temps['t2m'], Tref))

    else: # type == cooling
        if Tref == None: Tref = 295.15  # (22 C)
        dd = np.sum (xr.apply_ufunc( cooling_func_vec, max_temps['t2m'], min_temps['t2m'], mean_temps['t2m'], Tref))
    
    return dd.values

def calc_degreedays3(type, date_str, Tref=288.65):
    # Tref = 288.65 for 15.5 C
    # Tref = 291.15 for 18 C
    # Tref = 295.15 for 22 C
    # Uses the methodology from the paper 'European degree-day climatologies and trends for the period 1951 – 2011', 
    # J. Spinoni, J. Vogt, and P. Barbosa. INTERNATIONAL JOURNAL OF CLIMATOLOGY, 2014.

    if type not in ('heating','cooling'):
        print("WARNING: degree days type not understood. Defaulting to type='heating'.")
        type = 'heating'

    ds = get_temp_series() 
    month_ds = ds.sel(time=slice(*get_month_endpoints(date_str))) # this takes from midnight of first day to midnight of first day of following month
    max_temps = month_ds.resample(time='1D').max()['t2m'].values
    min_temps = month_ds.resample(time='1D').min()['t2m'].values
    mean_temps = (max_temps + min_temps) / 2    

    if type == 'heating':
        if Tref == None: Tref = 288.65 # (15.5 C)
        dd = heating_func_vec(max_temps, min_temps, mean_temps, Tref)
    else: # type == cooling
        if Tref == None: Tref = 295.15  # (22 C)
        dd = cooling_func_vec(max_temps, min_temps, mean_temps, Tref)

    return np.sum(dd)


    


In [2]:
ds = get_temp_series()
#month_ds = ds.sel(time=slice(get_month_endpoints(date_str))) # this takes from midnight of first day to midnight of first day of following month
#diff_time = month_ds['time'][-1] - month_ds['time'][0] # get time difference, saved as xarray with np.timedelta64 array in values
#%timeit calc_degreedays('heating','2020-04',Tref=288.65)
#%timeit calc_degreedays2('heating','2020-04',Tref=288.65)
print(calc_degreedays('heating','2020-05',Tref=288.65))
print(calc_degreedays2('heating','2020-05',Tref=288.65))
print(calc_degreedays3('heating','2020-05',Tref=288.65))

74.51414988835609
70.30045948028521
70.30045948028521


In [3]:
%timeit calc_degreedays('heating','2020-05',Tref=288.65)
%timeit calc_degreedays2('heating','2020-05',Tref=288.65)
%timeit calc_degreedays3('heating','2020-05',Tref=288.65)

619 ms ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
676 ms ± 13.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
673 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


TODO:
- can we do calculation for each month simultaneously for a given total range?
- look into only calculating degree days based on fixed hours of the day
- look into alternative methods of calculating degree days, ex. soft cutoff below Tref or thresholding to a few degrees below Tref


1. Make it work with arrays. If we want for example to select the degree days for an entire year, ideally we would run this calculation simultaneously in an array for each month instead of looping over everything.

2. Select daily hourly ranges. As you say, you want to look into only calculating degree days based on fixed hours of the day. So we (a) have to think about the implications of this and how this affects the meaning of degree days, and (b) modify the code to only integrate between those ranges of the day

3. Look into alternative methods of calculating degree days. For example, we can have a soft cutoff below the reference temperatre instead of just cutting it to 0 as we do now. Or also for example as Veit was saying, we can thresholding to a few degrees below the reference temperature so the degree days kick in once it is a few degrees colder than the reference temperature

4. What do we set the reference temperature to? We should have a default value, but also it may make sense to set it to thermostat settings? HOWEVER, the thermostat setting is taken into account into the heating consumption, whether it is weather corrected or not. Should the degree days weather correction not be independent of thermostat setting? Worth doing research on.

5. As in paper (put ref), we should set October to March as heating, and April to September as cooling. We only get monthly values still

Thoughts:

1. 